# WEBSCRAPPER TRF 1

## 1. Configurando o Ambiente

Faça download do Chromedriver download  
https://googlechromelabs.github.io/chrome-for-testing/#stable

Instale as dependencias

`` pip install -r requirements.txt `` 

ou pelo menos  
`` pip install selenium `` 

## 2. Usando o drive


### INICIANDO O DRIVE

````
driver = TRF5_Driver(service=s)

driver.login(
    username=os.environ.get("TRF1_login"), 
    password=os.environ.get("TRF1_password")
)
````

### MÉTODOS

#### IMPLICITLY WAIT  

Usar:  
`` driver.set_implicitly_wait(seconds) ``  

ao invés de:  
`` driver.implicitly_wait(seconds) `` 

Exemplo:  
``
driver.set_implicitly_wait(60)
``

#### CONSULTAR PROCESSOS
Consulta um processo e retorna um array de números
````
Espera um objeto com um ou mais desses atributos

consulta = {
    "nome": "NOME DA PARTE",
    "numero": "0000000-00.2024.4.01.0000",
    "cpf": "000.000.000-00"
}

driver.consultar_processo(consulta=consulta, wait_seconds=60)
````

Retorna 
`` ["000.000.000-00", "000.000.000-00"] ``

# Exemplo de uso

In [1]:
# Declarar a custom class TRF5_webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.alert import Alert
#from selenium.webdriver.support.ui import Select

from selenium import webdriver
from selenium.webdriver.common.by import By

class TRF5_Driver(webdriver.Chrome):
    
    implict_wait_value = 1
    loader_css = r"span#_viewRoot\:status\.start[style='display: none;']"
    
    def set_implicitly_wait(self, seconds):
        self.implict_wait_value = seconds
        self.implicitly_wait(seconds)
    
    # wait loader function
    def wait_loader(self, seconds=implict_wait_value):
        previous_implicit_wait = self.implict_wait_value
        self.implicitly_wait(1)
        i = 0
        while True:
            i+=1
            if i > seconds: break
            if self.find_elements(By.CSS_SELECTOR, self.loader_css):
                self.implicitly_wait(previous_implicit_wait)
                return
    
        self.implicitly_wait(previous_implicit_wait)
        raise Exception("Elemento nunca carregou")
    
    def login(self, username, password):

        self.get('https://pje1g.trf1.jus.br/pje')

        loginIframe = self.find_element(By.ID, "ssoFrame")
        self.switch_to.frame(loginIframe)
        self.find_element(By.ID, "username").send_keys(username)
        self.find_element(By.ID, "password").send_keys(password)
        self.find_element(By.CSS_SELECTOR, "input[type='submit']").click()

    def consultar_processo(self, consulta, wait_seconds=10):
        self.get("https://pje1g.trf1.jus.br/pje/Processo/ConsultaProcesso/listView.seam")
        
        if 'nome' in consulta.keys():
            self.find_element(By.CSS_SELECTOR, r"#fPP\:j_id145\:nomeParte").send_keys(consulta['nome'])
        
        if 'cpf' in consulta.keys():
            self.find_element(By.CSS_SELECTOR, r"#fPP\:dpDec\:documentoParte").send_keys(consulta['cpf'])

        if 'numero' in consulta.keys():
            self.find_element(By.CSS_SELECTOR, r"#fPP\:numeroProcesso\:numeroSequencial").send_keys(consulta['numero'][0:7])
            self.find_element(By.CSS_SELECTOR, r"#fPP\:numeroProcesso\:numeroDigitoVerificador").send_keys(consulta['numero'][8:10])
            self.find_element(By.CSS_SELECTOR, r"#fPP\:numeroProcesso\:Ano").send_keys(consulta['numero'][11:15])
            self.find_element(By.CSS_SELECTOR, r"#fPP\:numeroProcesso\:ramoJustica").send_keys(consulta['numero'][16])
            self.find_element(By.CSS_SELECTOR, r"#fPP\:numeroProcesso\:respectivoTribunal").send_keys(consulta['numero'][18:20])
            self.find_element(By.CSS_SELECTOR, r"#fPP\:numeroProcesso\:NumeroOrgaoJustica").send_keys(consulta['numero'][21:25])
            
        self.find_element(By.CSS_SELECTOR, r"#fPP\:searchProcessos").click()
        self.wait_loader(seconds=wait_seconds)
    
        rows = self.find_elements(By.CSS_SELECTOR, r"#fPP\:processosTable\:tb .rich-table-row")
        lawsuits = []
        for row in rows:
            cells = row.find_elements(By.CLASS_NAME, "rich-table-cell")
            lawsuits.append({ "processo": cells[1].text })
        
        return lawsuits

In [2]:
import os
import sys
import time
from dotenv import load_dotenv
load_dotenv()

project_path = os.path.dirname(os.path.abspath(sys.argv[0])).replace('.venv\\Lib\\site-packages', '')
chromedriver_path = f'{project_path}/chromedriver.exe'

s = Service(chromedriver_path)

default_implicitly_wait = 10

driver = TRF5_Driver(service=s)
driver.set_implicitly_wait(default_implicitly_wait)

driver.login(
    username=os.environ.get("TRF1_login"), 
    password=os.environ.get("TRF1_password")
)

In [ ]:
consulta = {
    "nome": "BERNADETE MARIA SANTOS BASTOS MORAES",
    "numero": "0004879-11.2016.4.01.3400",
    "cpf": "215.288.703-44"
}

processos = driver.consultar_processo(consulta=consulta, wait_seconds=120)
processos